In [ ]:
# Core libraries
import pandas as pd
import polars as pl

# Internal data access
from op_analytics.coreutils.duckdb_inmem.client import init_client, register_parquet_relation
from op_analytics.coreutils.clickhouse.oplabs import run_query_oplabs

# configurations
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import config

%load_ext autoreload
%autoreload 2

In [ ]:
CHAIN = config.CHAIN
BLOCK_DATE = config.BLOCK_DATE

In [ ]:
parquet_paths = [
    f"gs://oplabs-tools-data-sink/blockbatch/refined_traces/refined_transactions_fees_v2/chain=base/dt={BLOCK_DATE}/*.parquet"
]

# Load into DuckDB
ctx = init_client()
tx_fees_view = register_parquet_relation("refined_transactions_fees_v2", parquet_paths)

'op'